In [1]:
!nvidia-smi

Mon Apr 22 07:06:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              33W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title Setup environment { display-mode: 'form' }
%%capture
import os, subprocess, sys

src_dir = os.path.abspath("COMM061-NLP-CW")
first_install = False
if not os.path.exists(src_dir):
  output = subprocess.run(["git", "clone", "https://github.com/Cogniveon/COMM061-NLP-CW.git"], capture_output=True, check=True)
  assert output.returncode == 0
  print("Git clone:", output.stderr.decode('utf-8'))
  first_install = True

output = subprocess.run(
    ["git", "pull", "origin", "main"],
    cwd=src_dir,
    capture_output=True,
    check=True,
)
assert output.returncode == 0

print("Git pull:", output.stdout.decode('utf-8'))

already_up_to_date = 'Already up to date.' in output.stdout.decode('utf-8')

if first_install or not already_up_to_date:
  output = subprocess.run(
      ["pip", "install", "-e", src_dir],
      capture_output=True,
      check=True,
  )
  assert output.returncode == 0
  print("PIP install:", output.stdout.decode('utf-8'))
else:
  print("Skipping install...")

In [3]:
from nlp_cw import get_dataset, tokenize_dataset, init_trainer

dataset, id2label, label2id, label_list, num_labels = get_dataset(
    "surrey-nlp/PLOD-CW", ["B-O", "B-AC", "I-AC", "B-LF", "I-LF"]
)

dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1072
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 126
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 153
    })
})

In [4]:
# model_name = "romainlhardy/roberta-large-finetuned-ner"
model_name = "google-bert/bert-base-uncased"

tokenizer, tokenized_dataset = tokenize_dataset(dataset, model_name)

In [5]:
trainer = init_trainer(
    tokenizer,
    label_list,
    num_labels,
    id2label,
    label2id,
    batch_size=16,
    num_epochs=10,
    model_name_or_path=model_name,
    dataset=tokenized_dataset,
)
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.598900,0.253381,0.910079,0.911430,0.910754,0.906551
2,0.303100,0.226109,0.929827,0.915718,0.922719,0.918003
3,0.197200,0.225210,0.932966,0.922810,0.927861,0.923194
4,0.167100,0.231544,0.939799,0.926934,0.933322,0.927470
5,0.144200,0.244608,0.935734,0.934191,0.934962,0.930676
6,0.106200,0.250802,0.937583,0.931552,0.934558,0.929455
7,0.089100,0.261856,0.939721,0.933366,0.936533,0.931287
8,0.073800,0.276057,0.935138,0.934521,0.934829,0.929913
9,0.070300,0.284560,0.937231,0.933366,0.935295,0.930676
10,0.059600,0.286958,0.938650,0.933696,0.936167,0.931440


TrainOutput(global_step=670, training_loss=0.17096841246334474, metrics={'train_runtime': 345.2, 'train_samples_per_second': 31.054, 'train_steps_per_second': 1.941, 'total_flos': 689066401951200.0, 'train_loss': 0.17096841246334474, 'epoch': 10.0})

In [6]:
trainer.evaluate()

{'eval_loss': 0.2618560492992401,
 'eval_precision': 0.9397210229159748,
 'eval_recall': 0.9333663203034801,
 'eval_f1': 0.9365328920148945,
 'eval_accuracy': 0.9312872194228127,
 'eval_runtime': 1.5035,
 'eval_samples_per_second': 83.806,
 'eval_steps_per_second': 5.321,
 'epoch': 10.0}